In [1]:
import xarray as xr
import numpy as np

from utils import (
    detrend, monthly_anomalies, corr_with_ttest
)

In [2]:
t_grid_file = xr.open_zarr('data/ORCA025.L46-KFS006_TroPA.zarr/')

In [3]:
eq = dict(lat=0, lon=slice(140,260))

#### 1.1 D20 

In [4]:
d20_eq = monthly_anomalies(
    (-1) * t_grid_file.d20.sel(**eq)
).fillna(0.).rename("D20").compute()

#### 1.2 get SST data

In [5]:
sst_eq = monthly_anomalies(
    t_grid_file.sosstsst.sel(**eq)
).rename("SST")

In [6]:
ds = xr.Dataset({"SST": sst_eq, "D20": d20_eq})

In [7]:
ds_win = ds.rolling(time=21*12,center=True).construct("window_dim")

In [8]:
ds_windowed = detrend(
    ds_win.dropna('time').compute(),
    dim='window_dim',
)

In [9]:
corr, c_low, c_up = corr_with_ttest(
    ds_windowed.SST, ds_windowed.D20, dim="window_dim"
)

In [10]:
non_sig = xr.where(np.abs(corr) <= c_up, 1., np.nan)

## Write output

In [11]:
!rm -rf data_fig5S.zarr

xr.Dataset(
    dict(
        corr=corr,
        non_sig=non_sig,
    )
).to_zarr('data_fig5S.zarr');